In [18]:
import pandas as pd
import numpy as np
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [19]:
data = pd.read_csv('train.csv')
data.pop('id');
data_train, data_test = train_test_split(data, test_size = 0.1)


In [20]:
def getdata(data):
    X = []
    Y = []
    grouped = data.groupby('target')
    for key, item in grouped:
        tmp = grouped.get_group(key)
        X.append(tmp[tmp.columns[0:93]].values)    
        Y.append(tmp[tmp.columns[93]].values)
    return X,Y

In [21]:
def concatenate(X,Y):
    x = np.concatenate((X),axis = 0)
    y = np.concatenate((Y),axis = 0)
    return x,y 

In [22]:
def decision_tree(X,Y):
    from sklearn import tree
    dt = tree.DecisionTreeClassifier(min_samples_split= 20, random_state=99)
    dt.fit(X,Y)
    score = cross_val_score(dt, X, Y, cv = 5)
    return score,dt

In [23]:
def svm(X,Y):
    from sklearn import svm
    svm = svm.SVC()
    svm.fit(X, Y)
    score = cross_val_score(svm, X, Y, cv = 5)
    return score,svm

In [36]:
def xgboost(X,Y):
    import xgboost as xgb
    gbm = xgb.XGBClassifier(max_depth=20, n_estimators=300, learning_rate=0.05).fit(X, Y)
    score = cross_val_score(gbm, X, Y, cv = 5)
    return score,gbm

In [25]:
def logi_reg(X,Y):
    from sklearn import linear_model
    log = linear_model.LogisticRegression(C=1e5).fit(X,Y)
    score = cross_val_score(log, X, Y, cv = 5)
    return score,log

In [26]:
def random_forest(X,Y):
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(n_estimators = 100)
    forest = forest.fit(X,Y)
    score = cross_val_score(forest, X, Y, cv = 5)
    return score,forest

In [27]:
def bayes_naive_gaussien(X,Y):
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb = gnb.fit(X,Y)
    score = cross_val_score(gnb, X, Y, cv = 5)
    return score

In [28]:
def rate_correction (Y_pred,Y_true):
    rate = 0
    l = len(Y_pred)
    for i in range(l):
        if Y_pred[i] == Y_true[i]:
            rate = rate + 1
    return rate/l

In [29]:
def normalisation(X):
    for i in range(len(X)):
        X[i] = (X[i]-np.mean(X[i],axis=0))/np.std(X[i], axis=0)
    return X

In [30]:
X,Y = getdata(data)
X,Y = concatenate(X,Y)

In [31]:
score_dt  = decision_tree(X,Y)

In [32]:
score_dt 

(array([ 0.72127969,  0.71867173,  0.71800259,  0.72365869,  0.72152615]),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=20, min_weight_fraction_leaf=0.0,
             presort=False, random_state=99, splitter='best'))

In [33]:
score_log  = logi_reg(X,Y)

In [34]:
score_log

(array([ 0.75311036,  0.75187848,  0.75428248,  0.76010019,  0.75418317]),
 LogisticRegression(C=100000.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False))

In [37]:
score_xg = xgboost(X,Y)

In [38]:
score_xg

(array([ 0.82121506,  0.82459401,  0.82361021,  0.82514544,  0.82483227]),
 XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
        gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=20,
        min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
        objective='multi:softprob', reg_alpha=0, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1))

In [39]:
score_rf  = random_forest(X,Y)

In [40]:
score_rf

(array([ 0.80772338,  0.80560717,  0.80575307,  0.81334842,  0.81028211]),
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
             verbose=0, warm_start=False))

In [41]:
score_bn = bayes_naive_gaussien(X,Y)

In [42]:
score_bn

array([ 0.60631766,  0.61315343,  0.61595023,  0.61570782,  0.61733085])

In [43]:
score_svm  = svm(X,Y)

In [44]:
score_svm

(array([ 0.7851026 ,  0.78225741,  0.78595669,  0.79064318,  0.78465767]),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False))